In [1]:
import numpy as np
import graphinglib as gl
import pyregion
from scipy.optimize import curve_fit
import dill
from functools import partial

from src.statistics.advanced_stats import structure_function
from src.hdu.maps.map import Map

In [ ]:
def power_function(x, a, m):
    return a * x**m

def estimate_params(data, number_of_iterations: int=1000):
    sample_data = np.array([np.random.normal(mu, abs(sigma), number_of_iterations) for mu, sigma in data[:,1:3]])
    parameters = []
    for i in range(number_of_iterations):
        parameters.append(curve_fit(power_function, data[:,0], sample_data[:,i], [0.3, 0.5], maxfev=100000)[0])
    
    parameters_array = np.array(parameters)
    results = {
        "a": np.array([np.mean(parameters_array[:,0]), np.std(parameters_array[:,0])]),
        "m": np.array([np.mean(parameters_array[:,1]), np.std(parameters_array[:,1])]),
    }
    return results

In [ ]:
def get_plottables(data: np.ndarray, region_radius: float) -> list:
    sorted_data = data[np.argsort(data[:,0])]
    scatter = gl.Scatter(
        sorted_data[:,0],
        sorted_data[:,1],
        marker_size=3,
        face_color="black",
        # label="Structure function"
    )

    scatter.add_errorbars(
        y_error=np.abs(sorted_data[:,2]),
        errorbars_line_width=0.25,
        cap_width=0,
    )

    # gl_fit = gl.FitFromPolynomial(
    #     scatter.create_slice_x(0, region_radius),
    #     1,
    #     label="Fit (1st order polynomial)",
    #     line_width=1,
    #     color="orange",
    # )

    mask = (sorted_data[:,0] <= region_radius)
    pm = estimate_params(sorted_data[mask], 10000)
    fit = gl.Curve.from_function(
        func=partial(power_function, a=pm["a"][0], m=pm["m"][0]),
        x_min=0,
        x_max=region_radius,
        label=rf"Slope : {pm["m"][0]:.3f} ± {pm["m"][1]:.3f}",
        line_width=1,
        color="red",
    )

    return [scatter, fit]

In [ ]:
turbulence = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")

regions = [
    ("Sh2-158", None, 50),
    ("Diffuse region", pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"), 20),
    ("Central region", pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"), 10),
    ("Filament region", pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg"), 6)
]

### Structure functions of all regions

In [ ]:
figs = []

for name, region, region_radius in regions:
    # if not region:
    #     try:
    #         with open("applications/sh158/saved_data/str_func_turbulence_global.gz", "rb") as f:
    #             data = dill.load(f)
    #     except:
    #         data = structure_function(turbulence.get_masked_region(region).data)
    #         with open("applications/sh158/saved_data/str_func_turbulence_global.gz", "wb") as f:
    #             dill.dump(data, f)
    # else:
    data = structure_function(turbulence.get_masked_region(region).data)

    fig = gl.Figure(
        title=name,
        # x_label="Lag",
        # y_label="Structure function",
        x_lim=(0, region_radius*1.2),
    )
    plottables = get_plottables(data, region_radius)
    cropped_scatter = plottables[0].create_slice_x(0, region_radius*1.2)
    fig.y_lim = np.min(cropped_scatter.y_data)-0.2, np.max(cropped_scatter.y_data)+0.3
    fig.add_elements(*plottables)
    figs.append(fig)

multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
multifig.x_label = "Lag [pixels]"
multifig.y_label = "Structure Function [-]"
multifig.save("figures/sh158/advanced_stats/structure_functions_article_new.pdf")

Looping started
Looping finished
Regroupment started
Regroupment finished
Looping started
Looping finished
Regroupment started
Regroupment finished
Looping started
Looping finished
Regroupment started
Regroupment finished
Looping started
Looping finished
Regroupment started
Regroupment finished
